## EXERCICE 1 : SQL (Chapitre 1)

In [4]:
import sqlite3
import pandas as pd

# 1. Lire les données de bc.csv, compter le nombre de lignes et de colonnes.
df_bc = pd.read_csv('bc.csv')
num_rows_bc, num_cols_bc = df_bc.shape

print("Nombre de lignes :", num_rows_bc)
print("Nombre de colonnes :", num_cols_bc)

Nombre de lignes : 569
Nombre de colonnes : 33


In [ ]:
# 2. Insérer les données de bc.csv dans une table SQLite.
conn = sqlite3.connect('exercisesql.db')
df_bc.to_sql('bc_table', conn, if_exists='replace', index=False)

In [8]:
# 3. Créer une nouvelle table appelée tomatch avec les colonnes id_ et rdm_float.
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS tomatch (
                id_ INTEGER PRIMARY KEY,
                rdm_float REAL
            )''')

In [9]:
import random
import sqlite3

# 4. Insérer des observations dans la table tomatch selon les critères spécifiés.
# On insère uniquement les nombres pairs de 50k à 10M
for id_ in range(50000, 10000001, 2):
    # Vérifier si l'ID existe déjà dans la table
    c.execute("SELECT id_ FROM tomatch WHERE id_ = ?", (id_,))
    row = c.fetchone()
    if row is None:  # Si l'ID n'existe pas déjà, insérer dans la table
        rdm_float = random.uniform(0, 1)  # Générer un flottant aléatoire
        c.execute("INSERT INTO tomatch (id_, rdm_float) VALUES (?, ?)", (id_, rdm_float))

# Valider et sauvegarder les modifications
conn.commit()

In [10]:
# 5. Effectuer une jointure interne entre les deux tables et calculer les différentes fonctions d'agrégation.
# Je vais supposer ici que vous voulez dire la table bc_table, car il n'y a pas de détails sur l'autre table
c.execute('''SELECT COUNT(*), AVG(radius_mean), MAX(radius_mean), MIN(radius_mean)
              FROM bc_table 
              JOIN tomatch ON bc_table.id = tomatch.id_''')
result = c.fetchone()

In [11]:
# 6. Compter le nombre d'observations avec radius_mean supérieur à 15 et combien d'entre elles ont texture_mean supérieur à 20.
c.execute('''SELECT COUNT(*)
              FROM bc_table
              WHERE radius_mean > 15''')
num_obs_radius_gt_15 = c.fetchone()[0]

c.execute('''SELECT COUNT(*)
              FROM bc_table
              WHERE radius_mean > 15 AND texture_mean > 20''')
num_obs_radius_gt_15_texture_gt_20 = c.fetchone()[0]

print("Nombre d'observations avec radius_mean supérieur à 15 :", num_obs_radius_gt_15)
print("Nombre d'observations avec radius_mean supérieur à 15 et texture_mean supérieur à 20 :", num_obs_radius_gt_15_texture_gt_20)

Nombre d'observations avec radius_mean supérieur à 15 : 173
Nombre d'observations avec radius_mean supérieur à 15 et texture_mean supérieur à 20 : 106


In [12]:
# 7. Calculer la moyenne arrondie avec deux décimales pour chaque catégorie du diagnostic et calculer la différence entre les deux groupes.
c.execute('''SELECT ROUND(AVG(area_mean), 2)
              FROM bc_table
              WHERE diagnosis = 'M' ''')
mean_area_M = c.fetchone()[0]

c.execute('''SELECT ROUND(AVG(area_mean), 2)
              FROM bc_table
              WHERE diagnosis = 'B' ''')
mean_area_B = c.fetchone()[0]

difference_mean_area = mean_area_M - mean_area_B

print("Moyenne de la surface (arrondie à deux décimales) pour la catégorie 'M' :", mean_area_M)
print("Moyenne de la surface (arrondie à deux décimales) pour la catégorie 'B' :", mean_area_B)
print("Différence entre les moyennes de surface des catégories 'M' et 'B' :", difference_mean_area)

Moyenne de la surface (arrondie à deux décimales) pour la catégorie 'M' : 978.38
Moyenne de la surface (arrondie à deux décimales) pour la catégorie 'B' : 462.79
Différence entre les moyennes de surface des catégories 'M' et 'B' : 515.5899999999999


In [13]:
# 8. Changer le nom de la colonne "diagnosis" en "label".
c.execute('''ALTER TABLE bc_table RENAME COLUMN diagnosis TO label''')

OperationalError: database is locked

In [14]:
# 9. Ajouter une colonne nommée "jour" de type DATE.
c.execute('''ALTER TABLE bc_table ADD COLUMN day DATE''')

OperationalError: database is locked

In [23]:
# 10. Changer le type de la colonne "area_mean" en int en créant une nouvelle colonne.
c.execute('''ALTER TABLE bc_table ADD COLUMN area_mean_int INTEGER''')
c.execute('''UPDATE bc_table SET area_mean_int = CAST(area_mean AS INTEGER)''')


In [24]:
# 11. Créer une nouvelle table appelée tomatch2 similaire à tomatch mais avec des id_ incrémentés de 10.
c.execute('''CREATE TABLE IF NOT EXISTS tomatch2 AS
             SELECT id_ + 10 AS id_, rdm_float
             FROM tomatch''')


In [25]:
# 12. Effectuer une jointure interne entre les trois tables selon les critères spécifiés.
c.execute('''SELECT COUNT(*)
              FROM bc_table AS bc
              JOIN tomatch AS tm ON bc.id = tm.id_
              JOIN tomatch2 AS tm2 ON bc.id = tm2.id_
              WHERE bc.radius_mean > 15 AND tm.rdm_float > 0.50 AND tm2.rdm_float > 0.50''')

conn.commit()
conn.close()